In [2]:
import spotipy
import pandas as pd
import math
from spotipy.oauth2 import SpotifyClientCredentials

pd.options.display.max_rows = 999

In [3]:
sp = spotipy.Spotify(
    client_credentials_manager=SpotifyClientCredentials(
        client_id='c3c6cbaa304c4c86b120e8d9603d44a4', 
        client_secret='b8819ea70b3142819602499ee139c24f'))

nirvana_uri = 'spotify:artist:6olE6TJLqED3rqDCT0FyPh'

## Collecting data from playlist Nirvana all songs (Original)

In [4]:
playlist_id = "spotify:playlist:5PCYPdeq31cVX5pOH2vkpV"

offset = 0
limit = 100

all_tracks = []

while(True):
    results = sp.playlist_tracks(playlist_id, offset=offset, limit=limit)
    
    tracks = results['items']
    
    for track in tracks:
        all_tracks.append(track['track'])
    
    if len(tracks) < limit:
        break
    
    offset += limit


print("Total tracks:", len(all_tracks))


Total tracks: 401


Excluding Home Recordings

In [5]:
all_tracks = all_tracks[43:]
print("Total tracks:", len(all_tracks))
playlist_1 = all_tracks

Total tracks: 358


## Collecting data from playlist Nirvana Best Of

In [6]:
playlist_id = "spotify:playlist:7ETnL4fbHNcIdV1JmRuCZd"

offset = 0
limit = 100

all_tracks = []

while(True):
    results = sp.playlist_tracks(playlist_id, offset=offset, limit=limit)
    
    tracks = results['items']
    for track in tracks:
        all_tracks.append(track['track'])
    
    if len(tracks) < limit:
        break
    
    offset += limit

print("Total tracks:", len(all_tracks))

Total tracks: 33


In [7]:
playlist_2 = all_tracks

## Collecting data from playlist This Is Nirvana

In [8]:
playlist_id = "spotify:playlist:37i9dQZF1DZ06evO3M0Fbi"

offset = 0
limit = 100

all_tracks = []

while(True):
    results = sp.playlist_tracks(playlist_id, offset=offset, limit=limit)
    
    tracks = results['items']
    
    for track in tracks:
        all_tracks.append(track['track'])
    
    if len(tracks) < limit:
        break
    
    offset += limit


print("Total tracks:", len(all_tracks))

Total tracks: 39


In [9]:
playlist_3 = all_tracks

## Collecting data from Nevermind album (original 1991 release)

In [10]:
album_id = "spotify:album:6okv1avxEgYSdc2JYy6ZEi"

all_tracks = []

while(True):
    results = sp.album_tracks(album_id)
    
    tracks = results['items']
    
    for track in tracks:
        all_tracks.append(sp.track(track['uri']))
    
    if len(tracks) < limit:
        break
    
    offset += limit


print("Total tracks:", len(all_tracks))

Total tracks: 12


In [11]:
nevermind = all_tracks

## Collecting data from In Utero album (original 1993 release)

In [12]:
album_id = "spotify:album:76lXykCRFf2Nlsn9CrzD3G"

all_tracks = []

while(True):
    results = sp.album_tracks(album_id)
    
    tracks = results['items']
    
    for track in tracks:
        all_tracks.append(sp.track(track['uri']))
    
    if len(tracks) < limit:
        break
    
    offset += limit


print("Total tracks:", len(all_tracks))

Total tracks: 12


In [13]:
inutero = all_tracks

### Merging data

In [14]:
final_list = playlist_1

for track in playlist_2:
    if track not in final_list:
        final_list.append(track)
for track in playlist_3:
    if track not in final_list:
        final_list.append(track)
for track in nevermind:
    if track not in final_list:
        final_list.append(track)
for track in inutero:
    if track not in final_list:
        final_list.append(track)

len(final_list)

416

### Transforming data into a dataset

In [15]:
tracks_dict = {}

track_names = []
album_names = []
album_types = []
duration = []
year = []
artists = []

acousticness = []
danceability = []
energy = []
instrumentalness = []
key = []
liveness = []
loudness = []
mode = []
speechiness = []
tempo = []
time_signature = []
valence = []

for track in final_list:
    audio = sp.audio_features(tracks=track['id'])
    
    track_names.append(track['name'])
    album_names.append(track['album']['name'])
    album_types.append(track['album']['album_type'])
    duration.append(math.floor(track['duration_ms']/1000))
    year.append(track['album']['release_date'][0:4])
    artists.append(''.join([artist['name'] for artist in track['artists']]))
        
    acousticness.append(audio[0]['acousticness'])
    danceability.append(audio[0]['danceability'])
    energy.append(audio[0]['energy'])
    instrumentalness.append(audio[0]['instrumentalness'])
    key.append(audio[0]['key'])
    liveness.append(audio[0]['liveness'])
    loudness.append(audio[0]['loudness'])
    mode.append(audio[0]['mode'])
    speechiness.append(audio[0]['speechiness'])
    tempo.append(audio[0]['tempo'])
    time_signature.append(audio[0]['time_signature'])
    valence.append(audio[0]['valence'])    
        
tracks_dict['track_name'] = track_names
tracks_dict['album_name'] = album_names
tracks_dict['album_type'] = album_types
tracks_dict['duration'] = duration
tracks_dict['release_year'] = year
tracks_dict['artists'] = artists

tracks_dict['acousticness'] = acousticness
tracks_dict['danceability'] = danceability
tracks_dict['energy'] = energy
tracks_dict['instrumentalness'] = instrumentalness
tracks_dict['key'] = key
tracks_dict['liveness'] = liveness
tracks_dict['loudness'] = loudness
tracks_dict['mode'] = mode
tracks_dict['speechiness'] = speechiness
tracks_dict['tempo'] = tempo
tracks_dict['time_signature'] = time_signature
tracks_dict['valence'] = valence

In [27]:
df = pd.DataFrame(tracks_dict)

#### Sledeće promene su napravljene zbog stavki navedenih u napomenama.

In [28]:
df.loc[df['album_name'] == 'Bleach (Deluxe Edition)','release_year'] = 2009

df.loc[df['album_name'] == 'Nevermind (Super Deluxe Edition)', 'release_year'] = 2011
df.loc[df['album_name'] == 'Nevermind (Deluxe Edition)', 'release_year'] = 2011

df.loc[df['album_name'] == 'In Utero - 20th Anniversary Super Deluxe', 'release_year'] = 2013
df.loc[df['album_name'] == 'In Utero - 20th Anniversary - Deluxe Edition', 'release_year'] = 2013
df.loc[df['album_name'] == 'In Utero - 20th Anniversary Remaster', 'release_year'] = 2013

df.loc[df['album_name'] == 'Live At The Paramount', 'release_year'] = 2011

In [29]:
df.loc[(df['album_name'] == 'Bleach') | (df['album_name'] == 'Nevermind') | (df['album_name'] == 'In Utero'), 'album_type'] = 'studio album'

df.loc[(df['album_name'] == 'MTV Unplugged In New York') 
| (df['album_name'] == 'From The Muddy Banks Of The Wishkah (Live)') 
| (df['album_name'] == 'Live at Reading')
| (df['album_name'] == 'Live At The Paramount')
| (df['album_name'] == 'Live And Loud'), 'album_type'] = 'live album'

df.loc[(df['album_name'] == 'Incesticide') 
| (df['album_name'] == 'Nirvana') 
| (df['album_name'] == 'Sliver - The Best Of The Box'), 'album_type'] = 'compilation album'

df.loc[(df['album_name'] == 'With The Lights Out - Box Set'), 'album_type'] = 'box set'

df.loc[df['album_type'] == 'album', 'album_type'] = 'rerelease'

#### Sledeće promene napravljene su zbog poklapanja podataka u različitim data setovima

In [30]:
# Capializing every word in every track and album title
df.loc[:,['track_name', 'album_name']] = df.loc[:,['track_name', 'album_name']].apply(lambda x: x.str.title())

In [31]:
# Cleaning album names

df.loc[df['album_name'] == 'Live At The Paramount', 'track_name'] = df.loc[df['album_name'] == 'Live At The Paramount', 'track_name'].apply(lambda x: x[:-29])
df.loc[df['album_name'] == 'Live At Reading', 'track_name'] = df.loc[df['album_name'] == 'Live At Reading', 'track_name'].apply(lambda x: x[:-23])
df.loc[df['album_name'] == 'Live And Loud', 'track_name'] = df.loc[df['album_name'] == 'Live And Loud', 'track_name'].apply(lambda x: x[:-14])
df.loc[261:321, 'album_name'] = df.loc[261:321, 'album_name'].str.split(' - ').str[0] # With The Lights Out
df.loc[244:260,'album_name'] = df.loc[244:260,'album_name'].str.rstrip(' (Live)') #  From The Muddy Banks

In [32]:
# Cleaning track names

df.loc[[141,145], 'track_name'] = df.loc[[141,145], 'track_name'].str.slice(0,-27) # Incecticide
df.loc[[142,143,144], 'track_name'] = df.loc[[142,144], 'track_name'].str.slice(0,-29) # Incecticide

df.loc[230:243, 'track_name'] = df.loc[230:243, 'track_name'].str.split(' - ').str[0] # Mtv Unplugged In New York

df.loc[244:260, 'track_name'] = df.loc[244:260, 'track_name'].str.split(' - ').str[0] # From The Muddy Banks

df.loc[261:321, 'track_name'] = df.loc[261:321, 'track_name'].str.split(' - ').str[0] # With The Lights Out

df.loc[321:343,'track_name'] = df.loc[321:343,'track_name'].str.split(' - ').str[0] # Sliver - The Best Of The Box

df.loc[344:357,'track_name'] = df.loc[344:357,'track_name'].str.split(' - ').str[0] # Nirvana

In [33]:
df.loc[143,'track_name'] = 'Molly\'S Lips'

#### Čuvanje podatka u .csv datoteku.

In [34]:
df.to_csv('nirvana_spotify.csv', index=False)
df.to_csv('nirvana_v2.csv', index=False)